# Frost quakes and soil mechanics
In this notebook we consider the thermomechanical processes within a cylindrical subdomain of a homogeneous soil plot with the top boundary condition being defined with observed data. **In case you have not used a Jupyter Notebook before**, I note that I have collapsed the sections dealing with different boundary conditions. You can open them by clicking the arrow that appears next to the header when hovering over it.

Point of observation $X$ is set as the unmoving origin and we consider the stresses subjected to it. Another boundary condition is provided ultimately by the bedrock, which is considered unmoving for the purposes of this work. We model the thermomechanics of a finite annulus of radius $ R $ around this point of observation to consider the stresses subjected to the point of reference when the temperature of the topsoil changes. The radius $ R $ of the study should be large enough to safely assume homogeneity for the sake of the study.

We assume that the characteristic length scale of the plot $L$ is $L \gg R$ and finite. Because of the assumed homogeneity, we can limit our considerations to a 2-dimensional cross section of the system along the vertical axis $z$. We also assume that this embedding is far enough from the boundary of the plot to leave the cylindrical symmetry approximately unbroken.

In the beginning of this notebook we consider the system with having a traction-free (which I call "open") boundary at $r=R$. After this we also consider a system with a fixed boundary (zero displacement, which I call "closed") at $r=R$. In the last part we expand the study with a consideration of a Robin boundary condition, where the respose of the surrounding bulk to the plot under study is modeled by a spring constant, which we vary.

In [ ]:
# Imports etc. Run this before executing anything. 
# Remember to change the PATH variable to your ogs path (in terminal you can figure this out by running `which ogs`)
# Run the cell by Shift + Enter

from ogs6py import ogs
import vtuIO
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gmsh
import sys
import pandas as pd
plt.rcParams.update({'font.size': 16})
#plt.style.use(['dark_background'])

PATH = '/home/jremes/ogs-r/bin' # CHANGE TO YOUR OGS EXECUTABLE PATH

def setupPlot(ax, fsize=16):
    #sns.set_palette("Paired",n_colors=10)
    ax.set_xlabel(r'$r$ / m', fontsize=fsize)
    ax.set_ylabel(r'$\sigma$ /Pa', fontsize=fsize)
    ax.grid()
    ax.legend()

## Figuring out the times to the frost quake events

### Date correspondence figured out just by calculating
Beginning date of the data is 01/07/2020
End date is 29/06/2023

Frost quake events and time between midnights
21/11/2022   20 952h
12/12/2022   21 456h
5/1/2023     22 032h
6/1/2023     22 056h
7/1/2023     22 080h
8/1/2023     22 104h
1/2/2023     22 680h
2/2/2023     22 704h
3/2/2023     22 728h
5/3/2023     23 448h
6/3/2023     23 472h
7/3/2023     23 496h
8/3/2023     23 520h
9/3/2023     23 544h

In [ ]:
events = np.array([20952,21456,22032,22056,22080,22104,22680,22704,22728,23448,23472,23496,23520,23544])
testdepths = np.array([0.1,2,4,6,8,10,12,14,16,18,20,22,24,25.9])

# Open B.C.s

In [ ]:
# This cell runs the simulation. The project (-prj) file defines your simulation parameters (look at the porous solid phase). 
# The second, commented line is if you want to chenge parameters before the run
# Remember to run cells in order and have the PATH filled correctly
# The .prj file contains all the relevant parameters (except the geometry, which is determined by the .geo file.)

model = ogs.OGS(INPUT_FILE="../Column_RectMesh_tahtela.prj", PROJECT_FILE="../Column_RectMesh_tahtela_Out.prj", ogs_mode="debug")
#model.replace_text(10, xpath="./media/medium/properties/property[name='specific_heat_capacity']/specific_latent_heat")
model.write_input()

print("Running simulation")
model.run_model(path=PATH) 

print("Getting results")
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)

In [ ]:
# Checking that topsoil temperature observations are reproduced
# Using Pandas, as dealing with csv is IMO simplest that way
# NOTE THE DATA PATH

import pandas as pd
import csv
df = pd.read_csv('/home/jremes/karhi/1D_thermo-mech/Notebook/Data/Tähtelä_temp.csv', sep=',') # CHANGE LOCATION
dfTemp = df.T.to_numpy()
fig, ax = plt.subplots(figsize=(24,12))
ax.plot(dfTemp[0] / 3600, dfTemp[1], label='observations, BC')

#OGS Results at 0.1cm and 1 cm below the surface (bottom of the plot is at 0 m, height of the column is 26 m
pts = {"pt0": (1.25,25.99,0.0), "pt1": (1.25,25.9,0.0)}

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
T_Kelv = pvd.read_time_series("temperature", pts=pts) #read the documentation for vtuIO
time = pvd.timesteps / 3600 # time in h
T_Cels = T_Kelv['pt0'] - 273.15

ax.plot(time, 0*T_Cels, linewidth=2, linestyle='dashed', color='black')
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='lightcoral', label='d=0.01m')
T_Cels = T_Kelv['pt1'] - 273.15
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='teal', label='d=0.1m')  

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Amanzi-ATS simulation data I got from Jarkko. These are not up to date, but it is still a comparison

df0 = pd.read_csv('/home/jremes/karhi/1D_thermo-mech/Notebook/Data/atsSoilT100cm.csv',sep=';')
dfATS=df0.T.to_numpy()
fig = plt.figure()
ax = plt.axes()
ax.plot(dfATS[0], dfATS[1])
ax.set_xlabel('$t$ / h', fontsize=18)
ax.set_ylabel('$T$ /°C', fontsize=18)
ax.grid()

In [ ]:
# Observational data from Tähtelä. I have a script to treat the CSV

df = pd.read_csv('/home/jremes/karhi/1D_thermo-mech/Notebook/Data/Tähtelä_temp_obs10cm.csv')
dfA=df.T.to_numpy()
fig = plt.figure()
ax = plt.axes()
ax.plot(dfA[0], dfA[1])
ax.set_xlabel('$t$ / h', fontsize=18)
ax.set_ylabel('$T$ /°C', fontsize=18)
ax.grid()

In [ ]:
# Comparing the ATS and OGS results with some of the observations

fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (1.25,25.9,0.0), "pt1": (1.25,25.0,0.0)}

results = {}
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels = T_Kelv['pt0'] - 273.15

ax.plot(time, 0*T_Cels, linewidth=2, linestyle='dashed', color='black')
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='lightcoral', label='d=0.1m')

T_Cels = T_Kelv['pt1'] - 273.15
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='teal', label='d=1.0m')  

ax.plot(dfA[0], dfA[1], linewidth=1, linestyle='dotted', color='indigo', label='observed 0.1m')
ax.plot(dfATS[0], dfATS[1], linewidth=1.5, linestyle='solid', color='darkslateblue', label='ATS at 1.0m')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
#Plotting $\Delta T$. Did not reveal much to me such as is. Maybe needs the event timing to reveal anything.

fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (1.25,25.9,0.0), "pt1": (1.25,25.0,0.0)}

results = {}
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h

T_Cels = T_Kelv['pt0'] - 273.15
ax.plot(time[20:], 0*T_Cels[20:], linewidth=2, linestyle='dashed', color='black') # Delta cannot start from the beginning. This worked fsr
ax.plot(time[2:], np.gradient(T_Cels)[2:], linewidth=3, linestyle='solid', color='lightcoral', label='d=0.1m')

T_Cels = T_Kelv['pt1'] - 273.15
ax.plot(time[2:], np.gradient(T_Cels)[2:], linewidth=3, linestyle='solid', color='teal', label='d=1.0m')  

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$\Delta T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Checking that the initial condition for the temperature profile is satisfied.
depths = [(0.0, i*0.1, 0) for i in range(0,260)]

temps = pvd.read_time_slice(0, "temperature")
temps2 = pvd.read_set_data(0, "temperature", pointsetarray=depths)

fig, ax1 = plt.subplots(figsize=(6,4))
ax1.plot(depths,temps2-273.15)
plt.show()

In [ ]:
# Looking at the stress (rr) and displacement (rr) 0.1 m under the surface close to the origin and at r = R
pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))
fig, ax2 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r=0.01m')
ax1.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='r=R')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma_{rr}$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

ax2.plot(time, displacement['pt0'][:,0], linewidth=3, linestyle='solid', label='r=0.01m')
ax2.plot(time, displacement['pt1'][:,0], linewidth=3, linestyle='solid', label='r=R')

ax2.set_xlabel('$t$ / h', fontsize=24)
ax2.set_ylabel('$u_{rr}$ /m', fontsize=24)
ax2.grid()
ax2.legend(fontsize=24)

In [ ]:
# Looking at the stress (rr) and displacement (rr) 0.1 m under the surface close to the origin and at r = R
pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))
fig, ax2 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r=0.01m')
ax1.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='r=R')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma_{rr}$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

ax2.plot(time, displacement['pt0'][:,0], linewidth=3, linestyle='solid', label='r=0.01m')
ax2.plot(time, displacement['pt1'][:,0], linewidth=3, linestyle='solid', label='r=R')

ax2.set_xlabel('$t$ / h', fontsize=24)
ax2.set_ylabel('$u_{rr}$ /m', fontsize=24)
ax2.grid()
ax2.legend(fontsize=24)

In [ ]:
# Looking at the stress (rr) and displacement (rr) 0.1m under the surface close to the origin and at r = R 
# and comparing it to the temperature time series
pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r=0.01m')
ax1.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='r=R')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma_{rr}$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels = T_Kelv['pt0'] - 273.15

ax3 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax3.set_ylabel('$T$/°C', color='lightcoral', fontsize=24)
ax3.set_ylim(bottom=-60, top=25)
#ax3.plot(time, 0*T_Cels, linewidth=2, linestyle='dashed', color='black')
ax3.plot(time, T_Cels, linewidth=3, linestyle='solid', color='lightcoral', label='d=0.1m')
ax3.tick_params(axis='y', labelcolor='lightcoral')

fig.tight_layout()
plt.show()

In [ ]:
# Looking at the stress (rr) 0.1m under the surface close to the origin and at r = R 
# and comparing it to $\Delta T$ (equiv to the derivative at equal time slices)
pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r=0.01m')
ax1.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='r=R')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels = T_Kelv['pt0'] - 273.15

ax3 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax3.set_ylabel('$\Delta T$/°C', color='lightcoral', fontsize=24)
ax3.set_ylim(bottom=-15, top=15)
#ax3.plot(time, 0*T_Cels, linewidth=2, linestyle='dashed', color='black')
ax3.plot(time[2:], np.gradient(T_Cels[2:]), linewidth=3, linestyle='solid', color='lightcoral', label='d=0.1m')
ax3.tick_params(axis='y', labelcolor='lightcoral')

fig.tight_layout()
plt.show()

In [ ]:
# Comparing the stresses and displacements in the rr and the zz directions and displacement (rr) 0.1m under the surface close to the origin 
# Because we have a no-vertical-displacement -boundary condition, I am not sure if these are strictly comparable.
pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot0_zz = sigma_S['pt0'][:,1] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,1])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='$\sigma_{rr}$')
ax1.plot(time, sigma_tot0_zz, linewidth=3, linestyle='solid', label='$\sigma_{zz}$')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

time = pvd.timesteps / 3600 # time in h

fig.tight_layout()
plt.show()

In [ ]:
# Comparing the stress and ice content 0.1m belw the surface to see if freezing affects the pressure initially

pts = {"pt0": (1.3,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
phi = phi_I['pt0']
sigma_scaled_rr = (sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0]))*10**(-7)

time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_scaled_rr, linewidth=3, linestyle='solid', label='sigma')
ax1.plot(time, phi, linewidth=3, linestyle='solid', label='phi')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

fig.tight_layout()
plt.show()

In [ ]:
# Creating a dictionary for a timeseries in the next cell
indices = list(range(27))
rvals = list(map(lambda x: x / 10, indices))
rvals[0] += 0.01 # We cannot look at the boundary value as such.
keys = ["pt" + str(i) for i in range(27)]
coords = {}
for key, value in zip(keys, rvals):
    coords[key] = (value, 25.9, 0.0)

In [ ]:
# Checking the displacement along the r-axis at the depth of 0.1 m as a timeseries
import matplotlib.colors as mc

pts = {(0.01,25.9,0.0),(2.6,25.9,0.0)}

displacement = pvd.read_time_series("displacement", pts=coords)
time = pvd.timesteps / 3600 # time in h
col_number = len(indices)

fig, ax = plt.subplots(figsize=(24,12))

ax.plot(time, displacement[keys[1]][:,0], linewidth=3, linestyle='solid', label='disp')
ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('displacement /m', fontsize=24)
plt.show()

In [ ]:
# Settings Beginning date of the data is 01/07/2020 End date is 29/06/2023
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
col_number = len(indices)
start = '7/1/2020 00:00:00'
end = '6/29/2023 00:00:00'

# prepare a data frame
index = pd.date_range(start=start, end=end,  freq="12h")
columns = [f'y{i}' for i in range(col_number)] 
df = pd.DataFrame(index=index, columns=columns)
displacement = pvd.read_time_series("displacement", pts=coords)

# fill in the data
for n, col in enumerate(df.columns):
     df[col] = np.array([displacement[keys[n]][:,0][i] for i in range(len(df))])

# drawing a heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

ax1.plot(df)
#ax1.legend(df.columns)

ax2.imshow(df.T, aspect='auto', cmap='plasma')
#ax2.set_yticks(range(len(df.columns)))
#ax2.set_yticklabels(df.columns)

plt.show()

## THE SIGN DISPARITY IS OF NOTE

In [ ]:
# Settings Beginning date of the data is 01/07/2020 End date is 29/06/2023
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
col_number = len(indices)
start = '7/1/2020 00:00:00'
end = '6/29/2023 00:00:00'

# prepare a data frame
index = pd.date_range(start=start, end=end,  freq="12h")
columns = [f'y{i}' for i in range(col_number)] 
df = pd.DataFrame(index=index, columns=columns)

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigma_S = pvd.read_time_series("sigma", pts=coords)
sigma_I = pvd.read_time_series("sigma_ice", pts=coords)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=coords)

# fill in the data
for n, col in enumerate(df.columns):
     df[col] = np.array([((sigma_S[keys[n]][:,0] + np.multiply(phi_I[keys[n]],sigma_I[keys[n]][:,0]))*10**(-7))[i] for i in range(len(df))])

# drawing a heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

ax1.plot(df)
#ax1.legend(df.columns)

ax2.imshow(df.T, aspect='auto', cmap='plasma')
#ax2.set_yticks(range(len(df.columns)))
#ax2.set_yticklabels(df.columns)

plt.show()

In [ ]:
# Temperature at different depths. Again

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (0.01,25.9,0.0), "pt1": (0.01,25.7,0.0), "pt2": (0.01,25.5,0.0), "pt3": (0.01,25.2,0.0)}
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels0 = T_Kelv['pt0'] - 273.15
T_Cels1 = T_Kelv['pt1'] - 273.15
T_Cels2 = T_Kelv['pt2'] - 273.15
T_Cels3 = T_Kelv['pt3'] - 273.15

ax.plot(time, 0*T_Cels0, linewidth=2, linestyle='dashed', color='black')
ax.plot(time, T_Cels0, linewidth=3, linestyle='solid', label='d=0.1m')
ax.plot(time, T_Cels1, linewidth=3, linestyle='solid', label='d=0.3m')
ax.plot(time, T_Cels2, linewidth=3, linestyle='solid', label='d=0.5m')
ax.plot(time, T_Cels3, linewidth=3, linestyle='solid', label='d=0.8m')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Exporting some data!

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
pts = {"pt0": (0.01,25.9,0.0), "pt1": (0.01,25.7,0.0), "pt2": (0.01,25.5,0.0), "pt3": (0.01,25.2,0.0)}
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels0 = T_Kelv['pt0'] - 273.15
T_Cels1 = T_Kelv['pt1'] - 273.15
T_Cels2 = T_Kelv['pt2'] - 273.15
T_Cels3 = T_Kelv['pt3'] - 273.15

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr0 = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
#sigma_tot_zz0 = sigma_S['pt0'][:,1] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,1])
sigma_tot_rr1 = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])
#sigma_tot_zz1 = sigma_S['pt1'][:,1] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,1])
sigma_tot_rr2 = sigma_S['pt2'][:,0] + np.multiply(phi_I['pt2'],sigma_I['pt2'][:,0])
#sigma_tot_zz2 = sigma_S['pt2'][:,1] + np.multiply(phi_I['pt2'],sigma_I['pt2'][:,1])
sigma_tot_rr3 = sigma_S['pt3'][:,0] + np.multiply(phi_I['pt3'],sigma_I['pt3'][:,0])
#sigma_tot_zz3 = sigma_S['pt3'][:,1] + np.multiply(phi_I['pt3'],sigma_I['pt3'][:,1])

T_data = np.c_[time, T_Cels0, T_Cels1, T_Cels2, T_Cels3, sigma_tot_rr0, sigma_tot_rr1, sigma_tot_rr2, sigma_tot_rr3]

np.savetxt('Tähtelä_OGSdata_openBC.csv',T_data, delimiter=",")

In [ ]:
# Exporting some more data!

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
pts = {"pt0": (0.01,25.9,0.0), "pt1": (0.01,25.7,0.0), "pt2": (0.01,25.5,0.0), "pt3": (0.01,25.2,0.0)}
time = pvd.timesteps / 3600 # time in h
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
phiI0 = phi_I['pt0']
phiI1 = phi_I['pt1']
phiI2 = phi_I['pt2']
phiI3 = phi_I['pt3']

T_data = np.c_[time, phiI0, phiI1, phiI2, phiI3]

np.savetxt('Tähtelä_OGSdata_openBC_icevolumefraction.csv',T_data, delimiter=",")

In [ ]:
# Here we look at the temperature profile along the z-axis 0.5 m from the origin when a frost quake occurs
# events = the times at which we get frost quakes happening in the data. There's a conversion table in the beginning

events = np.array([20952,21456+1200,22032,22056,22080,22104,22680,22704,22728,23448,23472,23496,23520,23544])*3600 # in seconds
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
zaxis =  [(0.5,i,0) for i in np.linspace(start=0, stop=26, num=100)]
zaxisinv =  [(0.5,26-i,0) for i in np.linspace(start=0, stop=26, num=100)]
temps = {}
for i in range(0,13):
    temps[i] = pvd.read_set_data(events[i], "temperature", pointsetarray=zaxis)-273.15
fig, ax1 = plt.subplots(figsize=(24,12))
for i in range(0,13):
    ax1.plot(zaxisinv,temps[i])
plt.show()

In [ ]:
# Here we look at the stress profile as a function of depth when a frost quake occurs

import numpy as np
events = np.array([20952,21456+1200,22032,22056,22080,22104,22680,22704,22728,23448,23472,23496,23520,23544])*3600
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigmarr = {}; disps ={}; phiIs={}; 
for i in range(0,13):
    sigma_S = pvd.read_set_data(events[i],"sigma", pointsetarray=zaxis)
    sigma_I = pvd.read_set_data(events[i], "sigma_ice", pointsetarray=zaxis)
    phi_I = pvd.read_set_data(events[i], "ice_volume_fraction", pointsetarray=zaxis)
    sigma_tot0_rr = sigma_S[:,0] + np.multiply(phi_I,sigma_I[:,0])
    sigma_tot0_zz = sigma_S[:,1] + np.multiply(phi_I,sigma_I[:,1])

    displacement = pvd.read_set_data(events[i],"displacement", pointsetarray=zaxis)
    sigmarr[i] = sigma_tot0_rr
    disps[i] = displacement
    phiIs[i] = phi_I
fig, ax1 = plt.subplots(figsize=(24,12))
for i in range(0,13):
    ax1.plot(zaxisinv, sigmarr[i])
plt.show()

In [ ]:
# Frost quake event data export

events = np.array([20952,21456,22032,22056,22080,22104,22680,22704,22728,23448,23472,23496,23520,23544])*3600 #n=13
depths = [(0.01, i*2., 0) for i in range(0,13)]
depth = [i*2 for i in range(0,13)]
ind = 9

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
temps = pvd.read_set_data(events[ind], "temperature", pointsetarray=depths)-273.15

sigma_S = pvd.read_set_data(events[ind], "sigma", pointsetarray=depths)
sigma_I = pvd.read_set_data(events[ind], "sigma_ice", pointsetarray=depths)
phi_I = pvd.read_set_data(events[ind], "ice_volume_fraction", pointsetarray=depths)
sigma_tot_rr = sigma_S[:,0] + np.multiply(phi_I,sigma_I[:,0])
sigma_tot_zz = sigma_S[:,1] + np.multiply(phi_I,sigma_I[:,1])


T_data = np.c_[temps, sigma_tot_rr, sigma_tot_zz, phi_I]

np.savetxt('Tähtelä_OGSdata_openBC_event_'+str(ind)+'.csv',T_data, delimiter=",")

Frost quake events and time between midnights
0   21/11/2022   20 952h
1   12/12/2022   21 456h
2   5/1/2023     22 032h
3   6/1/2023     22 056h
4   7/1/2023     22 080h
5   8/1/2023     22 104h
6   1/2/2023     22 680h
7   2/2/2023     22 704h
8   3/2/2023     22 728h
9   5/3/2023     23 448h
10  6/3/2023     23 472h
11  7/3/2023     23 496h
12  8/3/2023     23 520h
13  9/3/2023     23 544h

In [ ]:
# ATS data

df = pd.read_csv('/home/jremes/karhi/1D_thermo-mech/Notebook/Data/FQ_events ice cappres soiltemp(21112022).csv', sep=';')
dfTemp = df[['depth','temp']].T.to_numpy()
ax = plt.axes()
ax.plot(dfTemp[0], dfTemp[1])
ax.set_xlabel('$depth$ / m', fontsize=18)
ax.set_ylabel('$T$ /°C', fontsize=18)
ax.grid()

In [ ]:
# Compare events to ATS. The temperature profiles are off and that's strange, since they should agree. OGS data has not changed, so maybe the problem is with the ATS data

ind=1
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
depth = [i*2 for i in range(0,13)]
temps = pvd.read_set_data(events[ind], "temperature", pointsetarray=depths)-273.15

sigma_S = pvd.read_set_data(events[ind], "sigma", pointsetarray=depths)
sigma_I = pvd.read_set_data(events[ind], "sigma_ice", pointsetarray=depths)
phi_I = pvd.read_set_data(events[ind], "ice_volume_fraction", pointsetarray=depths)
sigma_tot_rr = sigma_S[:,0] + np.multiply(phi_I,sigma_I[:,0])
sigma_tot_zz = sigma_S[:,1] + np.multiply(phi_I,sigma_I[:,1])

ax = plt.axes()
ax.plot(depth[::-1],temps)
print(len(depth))
print(len(dfTemp))

## ATS DATA
#df = pd.read_csv('/home/jremes/Karhinkangas/FQ_events ice cappres soiltemp(21112022).csv', sep=';')
df = pd.read_csv('/home/jremes/karhi/1D_thermo-mech/Notebook/Data/FQ_events ice cappres soiltemp(12122022).csv', sep=';')
#dfTemp = df[['depth','temp']].T.to_numpy()
dfTemp = df['temp'].T.to_numpy()
depth = [i*2 for i in range(0,len(dfTemp))]
ax.plot(depth, dfTemp)
ax.set_xlabel('$depth$ / m', fontsize=18)
ax.set_ylabel('$T$ /°C', fontsize=18)
ax.grid()

## Open BC with different size domain (sensitivity testing)

In [ ]:
# Define scales
m=1.0

# Dimensions of the whole site
Lz=26*m
Lr=5.0*m

geoName = "smallMesh"

In [ ]:
# This is a meshing cell. Look for the gmsh python documentation and the OGS forums for clarification. I had some troubles with the Python implementation all the time, and .geo files with gmsh GUI were more reliable for some reason.

import gmsh
import sys

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1) #turn terminal messages on
gmsh.model.add(geoName)

# Defining points
p4 = gmsh.model.geo.addPoint(0.0, 0,0, 0.0)
p1 = gmsh.model.geo.addPoint(0.0, Lz, 0.0)
p2 = gmsh.model.geo.addPoint(Lr,  Lz, 0.0)
p3 = gmsh.model.geo.addPoint(Lr, 0.0, 0.0)

# Boundary lines
bl1 = gmsh.model.geo.addLine(p1, p2)
bl2 = gmsh.model.geo.addLine(p2, p3)
bl3 = gmsh.model.geo.addLine(p3, p4) 
bl4 = gmsh.model.geo.addLine(p4, p1) 

gmsh.model.geo.synchronize()

# Loops and surfaces
bl = gmsh.model.geo.addCurveLoop([bl1, bl2, bl3, bl4])
bp = gmsh.model.geo.addPlaneSurface([bl])

gmsh.model.geo.synchronize()
                                    
# Physical groups
dim = 1
Top = gmsh.model.addPhysicalGroup(dim, [bl1])
gmsh.model.setPhysicalName(dim, Top, "Top")

Right = gmsh.model.addPhysicalGroup(dim, [bl2])
gmsh.model.setPhysicalName(dim, Right, "Right")

Bottom = gmsh.model.addPhysicalGroup(dim, [bl3])
gmsh.model.setPhysicalName(dim, Bottom, "Bottom")

Left = gmsh.model.addPhysicalGroup(dim, [bl4])
gmsh.model.setPhysicalName(dim, Left, "Left")

Boundary = gmsh.model.addPhysicalGroup(dim, [bl])
gmsh.model.setPhysicalName(dim, Boundary, "Boundary")
gmsh.model.geo.synchronize()

dim = 2
Domain = gmsh.model.addPhysicalGroup(dim, [bp])
gmsh.model.setPhysicalName(dim, Domain, "Domain")
gmsh.model.geo.synchronize()

# Define background field
field = gmsh.model.mesh.field
field.add("MathEval", 1)
field.setString(1, "F", "0.02+0.98*Exp(2*(26-(y+2)))/(1+Exp(2*(26-(y+2))))") #NOTICE: IF YOU CHANGE depth FROM 26, CHANGE 26 HERE
field.setAsBackgroundMesh(1)
gmsh.model.geo.synchronize()

# Meshing algorithm
gmsh.option.setNumber("Mesh.Algorithm", 6)

gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2) # Generate a 2D mesh

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
   
gmsh.write("../Column_RectMesh_size.msh")

gmsh.finalize()


In [ ]:
# Generating VTU files from MSH

!msh2vtu ../Column_RectMesh_size.msh
!ls .. #check if the VTU files were generated properly

In [ ]:
cmd1 = PATH + "/identifySubdomains"
!cd .. && {cmd1} -m  Column_RectMesh_size_domain.vtu -s 1e-13 -f -- Column_RectMesh_size_domain.vtu Column_RectMesh_size_physical_group_Top.vtu Column_RectMesh_size_physical_group_Bottom.vtu

In [ ]:
# This just gives you info about the mesh and if it is broken. It will be.
!checkMesh ../Column_RectMesh_size_domain.vtu -pv

In [ ]:
# If the nodes are in the incorrect order, run this
!NodeReordering -i ../Column_RectMesh_size_domain.vtu -o ../Column_RectMesh_size_domain.vtu -m 1

In [ ]:
!checkMesh ../Column_RectMesh_size_domain.vtu -pv

In [ ]:
# Running the model

model = ogs.OGS(INPUT_FILE="../Column_RectMesh_tahtela_sizevariation.prj", PROJECT_FILE="../Column_RectMesh_tahtela_sizevariation_Out.prj", ogs_mode="debug")
#model.replace_text(10, xpath="./media/medium/properties/property[name='specific_heat_capacity']/specific_latent_heat")
model.write_input()

print("Running simulation")
model.run_model(path=PATH) 

print("Getting results")
pvd = vtuIO.PVDIO("Results_open_size.pvd", dim=2)

In [ ]:
## Checking that topsoil temperature observations are reproduced

import pandas as pd
import csv
df = pd.read_csv('/home/jremes/Downloads/Tähtelä_temp.csv', sep=',')
dfTemp = df.T.to_numpy()
fig, ax = plt.subplots(figsize=(24,12))
ax.plot(dfTemp[0] / 3600, dfTemp[1], label='observations, BC')


#OGS Results at 1cm below the surface
pts = {"pt0": (1.25,25.99,0.0), "pt1": (1.25,25.9,0.0)}

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels = T_Kelv['pt0'] - 273.15

ax.plot(time, 0*T_Cels, linewidth=2, linestyle='dashed', color='black')
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='lightcoral', label='d=0.01m')
T_Cels = T_Kelv['pt1'] - 273.15
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='teal', label='d=0.1m')  

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Looking at the stress (rr) and displacement (rr) 0.1m under the surface close to the origin and at r = R
pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))
fig, ax2 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r=0.01m')
ax1.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='r=R')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma_{rr}$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

ax2.plot(time, displacement['pt0'][:,0], linewidth=3, linestyle='solid', label='r=0.01m')
ax2.plot(time, displacement['pt1'][:,0], linewidth=3, linestyle='solid', label='r=R')

ax2.set_xlabel('$t$ / h', fontsize=24)
ax2.set_ylabel('$u_{rr}$ /m', fontsize=24)
ax2.grid()
ax2.legend(fontsize=24)

# Closed B.C.s

This is just the same as the open b.c. stuff, but with the .prj file changed

In [ ]:
model = ogs.OGS(INPUT_FILE="../Column_RectMesh_tahtela_closed.prj", PROJECT_FILE="../Column_RectMesh_tahtela_closed_Out.prj", ogs_mode="debug")
#model.replace_text(10, xpath="./media/medium/properties/property[name='specific_heat_capacity']/specific_latent_heat")
model.write_input()

print("Running simulation")
model.run_model(path=PATH) 

print("Getting results")
pvd = vtuIO.PVDIO("Results_closed.pvd", dim=2)

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (1.25,25.9,0.0), "pt1": (1.25,25.0,0.0)}

results = {}
pvd = vtuIO.PVDIO("Results_closed.pvd", dim=2)
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels = T_Kelv['pt0'] - 273.15

ax.plot(time, 0*T_Cels, linewidth=2, linestyle='dashed', color='black')
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='lightcoral', label='d=0.1m')

T_Cels = T_Kelv['pt1'] - 273.15
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='teal', label='d=1.0m')  

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Looking at the stress (rr) and displacement (rr) 0.1m under the surface close to the origin and at r = R
pts = {"pt0": (1.25,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))
fig, ax2 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot0_zz = sigma_S['pt0'][:,1] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,1])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r')
ax1.plot(time, sigma_tot0_zz, linewidth=3, linestyle='solid', label='z')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\\sigma$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

ax2.plot(time, displacement['pt0'][:,1], linewidth=3, linestyle='solid', label='r=0.01m')
ax2.plot(time, displacement['pt1'][:,1], linewidth=3, linestyle='solid', label='r=R')

ax2.set_xlabel('$t$ / h', fontsize=24)
ax2.set_ylabel('$u_{rr}$ /m', fontsize=24)
ax2.grid()
ax2.legend(fontsize=24)

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,12))

sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot0_zz = sigma_S['pt0'][:,1] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,1])

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r')
ax1.plot(time, sigma_tot0_zz, linewidth=3, linestyle='solid', label='z')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\\sigma$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,12))

#sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot0_zz = sigma_S['pt0'][:,1] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,1])

#ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r')
ax1.plot(time, sigma_tot0_zz, linewidth=3, linestyle='solid', label='z')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\\sigma$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

In [ ]:
fig, ax2 = plt.subplots(figsize=(24,12))

ax2.plot(time, displacement['pt0'][:,1], linewidth=3, linestyle='solid', label='r=0.01m')
ax2.set_xlabel('$t$ / h', fontsize=24)
ax2.set_ylabel('$u_{y}$ /m', fontsize=24)
ax2.grid()
ax2.legend(fontsize=24)

In [ ]:
# Looking at the stress (rr) and displacement (rr) 0.1m under the surface close to the origin and at r = R 
# and comparing it to the temperature time series
pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))
fig, ax2 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='r=0.01m')
ax1.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='r=R')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels = T_Kelv['pt0'] - 273.15

ax3 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax3.set_ylabel('$T$/°C', color='lightcoral', fontsize=24)
ax3.set_ylim(bottom=-60, top=25)
#ax3.plot(time, 0*T_Cels, linewidth=2, linestyle='dashed', color='black')
ax3.plot(time, T_Cels, linewidth=3, linestyle='solid', color='lightcoral', label='d=0.1m')
ax3.tick_params(axis='y', labelcolor='lightcoral')

ax2.plot(time, displacement['pt0'][:,0], linewidth=3, linestyle='solid', label='r=0.01m')
ax2.plot(time, displacement['pt1'][:,0], linewidth=3, linestyle='solid', label='r=R')

ax2.set_xlabel('$t$ / h', fontsize=24)
ax2.set_ylabel('$u_{rr}$ /m', fontsize=24)
ax2.grid()
ax2.legend(fontsize=24)

fig.tight_layout()
plt.show()

In [ ]:
# Temperature different between the boundary conditions. Should be zero.

pvd = vtuIO.PVDIO("Results_closed.pvd", dim=2)
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (0.01,25.2,0.0)} #, "pt1": (0.01,25.7,0.0), "pt2": (0.01,25.5,0.0), "pt3": (0.01,25.2,0.0)
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels2 = T_Kelv['pt0'] - 273.15
ax.plot(time, 0*T_Cels2, linewidth=2, linestyle='dashed', color='black')

pvd1 = vtuIO.PVDIO("Results_open.pvd", dim=2)
T_Kelv1 = pvd1.read_time_series("temperature", pts=pts)
time = pvd1.timesteps / 3600 # time in h
T_Cels21 = T_Kelv1['pt0'] - 273.15

ax.plot(time, T_Cels2-  T_Cels21, linewidth=3, linestyle='solid')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Temperature at different depths. Again

pvd = vtuIO.PVDIO("Results_closed.pvd", dim=2)
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (0.01,25.9,0.0), "pt1": (0.01,25.7,0.0), "pt2": (0.01,25.5,0.0), "pt3": (0.01,25.2,0.0)}
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
#T_Cels0 = T_Kelv['pt0'] - 273.15
#T_Cels1 = T_Kelv['pt1'] - 273.15
T_Cels2 = T_Kelv['pt2'] - 273.15
#T_Cels3 = T_Kelv['pt3'] - 273.15

ax.plot(time, 0*T_Cels0, linewidth=2, linestyle='dashed', color='black')
#ax.plot(time, T_Cels0, linewidth=3, linestyle='solid', label='d=0.1m')
#ax.plot(time, T_Cels1, linewidth=3, linestyle='solid', label='d=0.3m')
ax.plot(time, T_Cels2, linewidth=3, linestyle='solid', label='d=0.5m')
#ax.plot(time, T_Cels3, linewidth=3, linestyle='solid', label='d=0.8m')

pvd1 = vtuIO.PVDIO("Results_open.pvd", dim=2)
T_Kelv1 = pvd1.read_time_series("temperature", pts=pts)
time = pvd1.timesteps / 3600 # time in h
#T_Cels01 = T_Kelv1['pt0'] - 273.15
#T_Cels11 = T_Kelv1['pt1'] - 273.15
T_Cels21 = T_Kelv1['pt2'] - 273.15
#T_Cels31 = T_Kelv1['pt3'] - 273.15

#ax.plot(time, T_Cels01, linewidth=3, linestyle='solid', label='Open d=0.1m')
#ax.plot(time, T_Cels11, linewidth=3, linestyle='solid', label='Open d=0.3m')
ax.plot(time, T_Cels21, linewidth=3, linestyle='solid', label='Open d=0.5m')
#ax.plot(time, T_Cels31, linewidth=3, linestyle='solid', label='Open d=0.8m')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Comparison of the stresses between the boundary conditions at r = R

fig, ax = plt.subplots(figsize=(24,12))

pvd = vtuIO.PVDIO("Results_closed.pvd", dim=2)
pts = {"pt0": (0.1,25.9,0.0)} 
time = pvd.timesteps / 3600 # time in h
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
ax.plot(time, sigma_tot_rr, linewidth=3, linestyle='solid', label='constrained')

pvd1 = vtuIO.PVDIO("Results_open.pvd", dim=2)
time = pvd1.timesteps / 3600 # time in h
sigma_S1 = pvd1.read_time_series("sigma", pts=pts)
sigma_I1 = pvd1.read_time_series("sigma_ice", pts=pts)
phi_I1 = pvd1.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot1_rr = sigma_S1['pt0'][:,0] + np.multiply(phi_I1['pt0'],sigma_I1['pt0'][:,0])
ax.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='traction-free')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$\sigma$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Exporting the stresses
pvd = vtuIO.PVDIO("Results_closed.pvd", dim=2)
pts = {"pt0": (0.1,25.9,0.0),"pt1": (0.1,25.7,0.0),"pt2": (0.1,25.5,0.0),"pt3": (0.1,25.2,0.0)} 
time = pvd.timesteps / 3600 # time in h
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_totC0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_totC1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])
sigma_totC2_rr = sigma_S['pt2'][:,0] + np.multiply(phi_I['pt2'],sigma_I['pt2'][:,0])
sigma_totC3_rr = sigma_S['pt3'][:,0] + np.multiply(phi_I['pt3'],sigma_I['pt3'][:,0])

pvd1 = vtuIO.PVDIO("Results_open.pvd", dim=2)
time = pvd1.timesteps / 3600 # time in h
sigma_S1 = pvd1.read_time_series("sigma", pts=pts)
sigma_I1 = pvd1.read_time_series("sigma_ice", pts=pts)
phi_I1 = pvd1.read_time_series("ice_volume_fraction", pts=pts)
sigma_totO0_rr = sigma_S1['pt0'][:,0] + np.multiply(phi_I1['pt0'],sigma_I1['pt0'][:,0])
sigma_totO1_rr = sigma_S1['pt1'][:,0] + np.multiply(phi_I1['pt1'],sigma_I1['pt1'][:,0])
sigma_totO2_rr = sigma_S1['pt2'][:,0] + np.multiply(phi_I1['pt2'],sigma_I1['pt2'][:,0])
sigma_totO3_rr = sigma_S1['pt3'][:,0] + np.multiply(phi_I1['pt3'],sigma_I1['pt3'][:,0])

T_data = np.c_[time, sigma_totC0_rr, sigma_totC1_rr, sigma_totC2_rr, sigma_totC3_rr, sigma_totO0_rr, sigma_totO1_rr, sigma_totO2_rr, sigma_totO3_rr]

np.savetxt('Tähtelä_OGSdata_stresses.csv',T_data, delimiter=",")

In [ ]:
# Exporting data

pvd = vtuIO.PVDIO("Results_closed.pvd", dim=2)
pts = {"pt0": (0.01,25.9,0.0), "pt1": (0.01,25.7,0.0), "pt2": (0.01,25.5,0.0), "pt3": (0.01,25.2,0.0)}
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels0 = T_Kelv['pt0'] - 273.15
T_Cels1 = T_Kelv['pt1'] - 273.15
T_Cels2 = T_Kelv['pt2'] - 273.15
T_Cels3 = T_Kelv['pt3'] - 273.15

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr0 = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
#sigma_tot_zz0 = sigma_S['pt0'][:,1] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,1])
sigma_tot_rr1 = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])
#sigma_tot_zz1 = sigma_S['pt1'][:,1] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,1])
sigma_tot_rr2 = sigma_S['pt2'][:,0] + np.multiply(phi_I['pt2'],sigma_I['pt2'][:,0])
#sigma_tot_zz2 = sigma_S['pt2'][:,1] + np.multiply(phi_I['pt2'],sigma_I['pt2'][:,1])
sigma_tot_rr3 = sigma_S['pt3'][:,0] + np.multiply(phi_I['pt3'],sigma_I['pt3'][:,0])
#sigma_tot_zz3 = sigma_S['pt3'][:,1] + np.multiply(phi_I['pt3'],sigma_I['pt3'][:,1])

T_data = np.c_[time, T_Cels0, T_Cels1, T_Cels2, T_Cels3, sigma_tot_rr0, sigma_tot_rr1, sigma_tot_rr2, sigma_tot_rr3]

np.savetxt('Tähtelä_OGSdata_closedBC.csv',T_data, delimiter=",")

# Testing changing properties to study sign disparity

In [ ]:
# Some results for the sine wave test. These will serve as the top BC
tend = 165725333 #94435200
period = tend/(2*5*np.pi)
deltat = 43200
#repeats = 2186
times = np.arange(0, tend, deltat)
temps = np.matrix(273.15+20*np.sin(times/period))
np.savetxt('file.txt', np.matrix(temps), delimiter=' ', fmt='%1.2f')

In [ ]:
print(1657152/(432))

In [ ]:
model = ogs.OGS(INPUT_FILE="../Column_RectMesh_tahtela_test.prj", PROJECT_FILE="../Column_RectMesh_tahtela_Out.prj", ogs_mode="debug")
#model.replace_text(10, xpath="./media/medium/properties/property[name='specific_heat_capacity']/specific_latent_heat")
model.write_input()

print("Running simulation")
model.run_model(path=PATH) 

In [ ]:
# Comparison of the stresses between traction-free simulations with porosity of 0.01, 1.0 and with thermal expansivity of ice 1000x as well as solid matrix thermal expansivity x1000

fig, ax = plt.subplots(figsize=(24,12))
pts = {"pt0": (0.1,25.9,0.0)} 

pvd = vtuIO.PVDIO("Results_open_TEe3.pvd", dim=2)
time = pvd.timesteps / 3600 # time in h
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
ax.plot(time, sigma_tot_rr, linewidth=3, linestyle='solid', label='TE x 1000')

#pvd1 = vtuIO.PVDIO("Results_open_por0.pvd", dim=2)
#time = pvd1.timesteps / 3600 # time in h
#sigma_S1 = pvd1.read_time_series("sigma", pts=pts)
#sigma_I1 = pvd1.read_time_series("sigma_ice", pts=pts)
#phi_I1 = pvd1.read_time_series("ice_volume_fraction", pts=pts)
#sigma_tot1_rr = sigma_S1['pt0'][:,0] + np.multiply(phi_I1['pt0'],sigma_I1['pt0'][:,0])
#ax.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='porosity 0.01')

#pvd2 = vtuIO.PVDIO("Results_open_por1.pvd", dim=2)
#time = pvd2.timesteps / 3600 # time in h
#sigma_S2 = pvd2.read_time_series("sigma", pts=pts)
#sigma_I2 = pvd2.read_time_series("sigma_ice", pts=pts)
#phi_I2 = pvd2.read_time_series("ice_volume_fraction", pts=pts)
#sigma_tot2_rr = sigma_S2['pt0'][:,0] + np.multiply(phi_I2['pt0'],sigma_I2['pt0'][:,0])
#ax.plot(time, sigma_tot2_rr, linewidth=3, linestyle='solid', label='porosity 1.0')

#pvd3 = vtuIO.PVDIO("Results_open.pvd", dim=2)
#pts = {"pt0": (0.1,25.9,0.0)} 
#time = pvd3.timesteps / 3600 # time in h
#sigma_S3 = pvd3.read_time_series("sigma", pts=pts)
#sigma_I3 = pvd3.read_time_series("sigma_ice", pts=pts)
#phi_I3 = pvd3.read_time_series("ice_volume_fraction", pts=pts)
#sigma_tot3_rr = sigma_S3['pt0'][:,0] + np.multiply(phi_I3['pt0'],sigma_I3['pt0'][:,0])
#ax.plot(time, sigma_tot3_rr, linewidth=3, linestyle='solid', label='normal')

#pvd4 = vtuIO.PVDIO("Results_open_test.pvd", dim=2)
#pts = {"pt0": (0.1,25.9,0.0)} 
#time4 = pvd4.timesteps / 3600 # time in h
#sigma_S4 = pvd4.read_time_series("sigma", pts=pts)
#sigma_I4 = pvd4.read_time_series("sigma_ice", pts=pts)
#phi_I4 = pvd4.read_time_series("ice_volume_fraction", pts=pts)
#sigma_tot4_rr = sigma_S4['pt0'][:,0] + np.multiply(phi_I4['pt0'],sigma_I4['pt0'][:,0])
#ax.plot(time4, sigma_tot4_rr, linewidth=3, linestyle='solid', label='time delay')

#pvd5 = vtuIO.PVDIO("Results_open_solidTE.pvd", dim=2)
#time5 = pvd5.timesteps / 3600 # time in h
#sigma_S5 = pvd5.read_time_series("sigma", pts=pts)
#sigma_I5 = pvd5.read_time_series("sigma_ice", pts=pts)
#phi_I5 = pvd5.read_time_series("ice_volume_fraction", pts=pts)
#sigma_tot5_rr = sigma_S5['pt0'][:,0] + np.multiply(phi_I5['pt0'],sigma_I5['pt0'][:,0])
#ax.plot(time5, sigma_tot5_rr, linewidth=3, linestyle='solid', label='solid TE x1000')

pvd6 = vtuIO.PVDIO("Results_open_TEe3_fixBCs.pvd", dim=2)
time6 = pvd6.timesteps / 3600 # time in h
sigma_S6 = pvd6.read_time_series("sigma", pts=pts)
sigma_I6 = pvd6.read_time_series("sigma_ice", pts=pts)
phi_I6 = pvd6.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr6 = sigma_S6['pt0'][:,0] + np.multiply(phi_I6['pt0'],sigma_I6['pt0'][:,0])
ax.plot(time6, sigma_tot_rr6, linewidth=3, linestyle='solid', label='TE x 1000, switched BCs')

pvd7 = vtuIO.PVDIO("Results_open_TEe3_fixBCs_k1.pvd", dim=2)
time7 = pvd7.timesteps / 3600 # time in h
sigma_S7 = pvd7.read_time_series("sigma", pts=pts)
sigma_I7 = pvd7.read_time_series("sigma_ice", pts=pts)
phi_I7 = pvd7.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr7 = sigma_S7['pt0'][:,0] + np.multiply(phi_I7['pt0'],sigma_I7['pt0'][:,0])
ax.plot(time7, sigma_tot_rr7, linewidth=3, linestyle='solid', label='TE x 1000, switched BCs, k=1')

pvd8 = vtuIO.PVDIO("Results_open_TEe3_fixBCs_k2.pvd", dim=2)
time8 = pvd8.timesteps / 3600 # time in h
sigma_S8 = pvd8.read_time_series("sigma", pts=pts)
sigma_I8 = pvd8.read_time_series("sigma_ice", pts=pts)
phi_I8 = pvd8.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr8 = sigma_S8['pt0'][:,0] + np.multiply(phi_I8['pt0'],sigma_I8['pt0'][:,0])
ax.plot(time8, sigma_tot_rr8, linewidth=3, linestyle='solid', label='TE x 1000, switched BCs, k=2')

pvd9 = vtuIO.PVDIO("Results_open_TEe3_fixBCs_k2_closed.pvd", dim=2)
time9 = pvd9.timesteps / 3600 # time in h
sigma_S9 = pvd9.read_time_series("sigma", pts=pts)
sigma_I9 = pvd9.read_time_series("sigma_ice", pts=pts)
phi_I9 = pvd9.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr9 = sigma_S9['pt0'][:,0] + np.multiply(phi_I9['pt0'],sigma_I9['pt0'][:,0])
ax.plot(time9, sigma_tot_rr9, linewidth=3, linestyle='solid', label='TE x 1000, switched BCs, k=2, radially constrained')

pvd10 = vtuIO.PVDIO("Results_open_TEe3_fixBCs_k2_Rx4.pvd", dim=2)
time10 = pvd10.timesteps / 3600 # time in h
sigma_S10 = pvd10.read_time_series("sigma", pts=pts)
sigma_I10 = pvd10.read_time_series("sigma_ice", pts=pts)
phi_I10 = pvd10.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot_rr10 = sigma_S10['pt0'][:,0] + np.multiply(phi_I10['pt0'],sigma_I10['pt0'][:,0])
ax.plot(time10, sigma_tot_rr10, linewidth=3, linestyle='solid', label='TE x 1000, switched BCs, k=2, R x 4')
ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$\\sigma$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

plt.savefig('stress.png')

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pvd = vtuIO.PVDIO("Results_open_TEe3_k1.pvd", dim=2)
pts = {"pt0": (1.25,25.9,0.0)} 
time = pvd.timesteps / 3600 # time in h
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_ice = np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot_rr = sigma_S['pt0'][:,0] + sigma_ice
ax.plot(time, sigma_S['pt0'][:,0], linewidth=3, linestyle='solid', label='solid')
ax.plot(time, sigma_ice, linewidth=3, linestyle='solid', label='ice')
ax.plot(time, sigma_tot_rr, linewidth=3, linestyle='solid', label='total')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$\sigma$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Comparison of the displacements between traction-free simulations with porosity of 0.01, 1.0 and with thermal expansivity of ice 1000x

fig, ax = plt.subplots(figsize=(24,12))

#pvd = vtuIO.PVDIO("Results_open_TEe3.pvd", dim=2)
pts = {"pt0": (1.25,25.9,0.0)} 
time = pvd.timesteps / 3600 # time in h
dis = pvd.read_time_series("displacement", pts=pts)
ax.plot(time, dis['pt0'][:,0], linewidth=3, linestyle='solid', label='TE x 1000')

##pvd1 = vtuIO.PVDIO("Results_open_por0.pvd", dim=2)
#time = pvd1.timesteps / 3600 # time in h
#dis1 = pvd1.read_time_series("displacement", pts=pts)
#ax.plot(time, dis1['pt0'][:,0], linewidth=3, linestyle='solid', label='porosity 0.01')

##pvd2 = vtuIO.PVDIO("Results_open_por1.pvd", dim=2)
#time = pvd2.timesteps / 3600 # time in h
#dis2 = pvd2.read_time_series("displacement", pts=pts)
#ax.plot(time, dis2['pt0'][:,0], linewidth=3, linestyle='solid', label='porosity 1.0')

##pvd3 = vtuIO.PVDIO("Results_open.pvd", dim=2)
#time = pvd3.timesteps / 3600 # time in h
#dis3 = pvd3.read_time_series("displacement", pts=pts)
#ax.plot(time, dis3['pt0'][:,0], linewidth=3, linestyle='solid', label='normal')

##pvd4 = vtuIO.PVDIO("Results_open_test.pvd", dim=2)
#time4 = pvd4.timesteps / 3600 # time in h
#dis4 = pvd4.read_time_series("displacement", pts=pts)
#ax.plot(time4, dis4['pt0'][:,0], linewidth=3, linestyle='solid', label='delayed time')

##pvd5 = vtuIO.PVDIO("Results_open_solidTE.pvd", dim=2)
#time5 = pvd5.timesteps / 3600 # time in h
#dis5 = pvd5.read_time_series("displacement", pts=pts)
#ax.plot(time5, dis5['pt0'][:,0], linewidth=3, linestyle='solid', label='solid TE x 1000')

time6 = pvd6.timesteps / 3600 # time in h
dis6 = pvd6.read_time_series("displacement", pts=pts)
ax.plot(time6, dis6['pt0'][:,0], linewidth=3, linestyle='solid', label='TE x 1000, fixed BCs')

time7 = pvd7.timesteps / 3600 # time in h
dis7 = pvd7.read_time_series("displacement", pts=pts)
ax.plot(time7, dis7['pt0'][:,0], linewidth=3, linestyle='solid', label='TE x 1000, fixed BCs, k=1')

time8 = pvd8.timesteps / 3600 # time in h
dis8 = pvd8.read_time_series("displacement", pts=pts)
ax.plot(time8, dis8['pt0'][:,0], linewidth=3, linestyle='solid', label='TE x 1000, fixed BCs, k=2')

time9 = pvd9.timesteps / 3600 # time in h
dis9 = pvd9.read_time_series("displacement", pts=pts)
ax.plot(time9, dis9['pt0'][:,0], linewidth=3, linestyle='solid', label='TE x 1000, fixed BCs, k=2, constrained')

time10 = pvd10.timesteps / 3600 # time in h
dis10 = pvd10.read_time_series("displacement", pts=pts)
ax.plot(time10, dis10['pt0'][:,0], linewidth=3, linestyle='solid', label='TE x 1000, fixed BCs, k=2, R x 4')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('displacement /m', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

plt.savefig('displacement.png')

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pvd11 = vtuIO.PVDIO("Results_open_sine_k2_Rx4.pvd", dim=2)
time11 = pvd11.timesteps / 3600 # time in h
pts = {"pt0": (1.25,25.9,0.0)} 
dis11 = pvd11.read_time_series("displacement", pts=pts)
ax.plot(time11, dis11['pt0'][:,1], linewidth=3, linestyle='solid', label='Sine temp, fixed BCs, k=2')

pvd12 = vtuIO.PVDIO("Results_open_TEe3_sine_k2_Rx4.pvd", dim=2)
time12 = pvd12.timesteps / 3600 # time in h
dis12 = pvd12.read_time_series("displacement", pts=pts)
ax.plot(time12, dis12['pt0'][:,1], linewidth=3, linestyle='solid', label='Sine temp, TE x 1000, fixed BCs, k=2')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('displacement /m', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

plt.savefig('displacement.png')

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pvd11 = vtuIO.PVDIO("Results_open_sine_k2_Rx4.pvd", dim=2)
time11 = pvd11.timesteps / 3600 # time in h
pts = {"pt0": (1.25,25.9,0.0)} 
dis11 = pvd11.read_time_series("displacement", pts=pts)
ax.plot(time11, dis11['pt0'][:,0], linewidth=3, linestyle='solid', label='Sine temp, fixed BCs, k=2')

pvd12 = vtuIO.PVDIO("Results_open_TEe3_sine_k2_Rx4.pvd", dim=2)
time12 = pvd12.timesteps / 3600 # time in h
dis12 = pvd12.read_time_series("displacement", pts=pts)
ax.plot(time12, dis12['pt0'][:,0], linewidth=3, linestyle='solid', label='Sine temp, TE x 1000, fixed BCs, k=2')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('displacement /m', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

plt.savefig('displacement.png')

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (1.25,25.9,0.0)} 

sigma_S11 = pvd11.read_time_series("sigma", pts=pts)
sigma_I11 = pvd11.read_time_series("sigma_ice", pts=pts)
phi_I11 = pvd11.read_time_series("ice_volume_fraction", pts=pts)
sigma_ice11 = np.multiply(phi_I11['pt0'],sigma_I11['pt0'][:,1])
sigma_tot_rr11 = sigma_S11['pt0'][:,1] + sigma_ice11
ax.plot(time11, sigma_S11['pt0'][:,1], linewidth=3, linestyle='solid', label='solid')
ax.plot(time11, sigma_ice11, linewidth=3, linestyle='solid', label='ice')
ax.plot(time11, sigma_tot_rr11, linewidth=3, linestyle='solid', label='total')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$\\sigma$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

plt.savefig('partial_stress.png')

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (1.25,25.9,0.0)} 

sigma_S11 = pvd11.read_time_series("sigma", pts=pts)
sigma_I11 = pvd11.read_time_series("sigma_ice", pts=pts)
phi_I11 = pvd11.read_time_series("ice_volume_fraction", pts=pts)
sigma_ice11 = np.multiply(phi_I11['pt0'],sigma_I11['pt0'][:,0])
sigma_tot_rr11 = sigma_S11['pt0'][:,0] + sigma_ice11
ax.plot(time11, sigma_S11['pt0'][:,0], linewidth=3, linestyle='solid', label='solid')
ax.plot(time11, sigma_ice11, linewidth=3, linestyle='solid', label='ice')
ax.plot(time11, sigma_tot_rr11, linewidth=3, linestyle='solid', label='total')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$\\sigma$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

plt.savefig('partial_stress.png')

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pvd11 = vtuIO.PVDIO("Results_open_sine_k2_5cycles.pvd", dim=2)
time11 = pvd11.timesteps / 3600 # time in h
pts = {"pt0": (1.25,25.99,0.0)} 
temp11 = pvd11.read_time_series("temperature", pts=pts)
T_Cels1 = temp11['pt0'] - 273.15
ax.plot(time11, T_Cels1, linewidth=3, linestyle='solid', label='Sine temp, fixed BCs, k=2')

#pvd12 = vtuIO.PVDIO("Results_open_TEe3_sine_k2_Rx4.pvd", dim=2)
#time12 = pvd12.timesteps / 3600 # time in h
#temp12 = pvd12.read_time_series("temperature", pts=pts)
#T_Cels2 = temp12['pt0'] - 273.15
#ax.plot(time12, T_Cels2, linewidth=3, linestyle='solid', label='Sine temp, TE x 1000, fixed BCs, k=2')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('temperature /C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

plt.savefig('temp.png')

In [ ]:
vtuIO.PVDIO("Results_open_sine_k2_5cycles.pvd", dim=2)

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (1.25,25.9,0.0)} 

sigma_S11 = pvd11.read_time_series("sigma", pts=pts)
sigma_I11 = pvd11.read_time_series("sigma_ice", pts=pts)
phi_I11 = pvd11.read_time_series("ice_volume_fraction", pts=pts)
sigma_ice11 = np.multiply(phi_I11['pt0'],sigma_I11['pt0'][:,0])
sigma_tot_rr11 = sigma_S11['pt0'][:,0] + sigma_ice11
ax.plot(time11, sigma_tot_rr11, linewidth=3, linestyle='solid', label='Sine temp, fixed BCs, k=2')

sigma_S12 = pvd12.read_time_series("sigma", pts=pts)
sigma_I12 = pvd12.read_time_series("sigma_ice", pts=pts)
phi_I12 = pvd12.read_time_series("ice_volume_fraction", pts=pts)
sigma_ice12 = np.multiply(phi_I12['pt0'],sigma_I12['pt0'][:,0])
sigma_tot_rr12 = sigma_S12['pt0'][:,0] + sigma_ice12
ax.plot(time12, sigma_tot_rr12, linewidth=3, linestyle='solid', label='Sine temp, TE ice x 1000 fixed BCs, k=2')

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$\\sigma$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

plt.savefig('stress.png')

In [ ]:
# Creating a dictionary for a timeseries in the next cell
indices = list(range(27))
rvals = list(map(lambda x: x / 10, indices))
rvals[0] += 0.01 # We cannot look at the boundary value as such.
keys = ["pt" + str(i) for i in range(27)]
coords = {}
for key, value in zip(keys, rvals):
    coords[key] = (value, 25.9, 0.0)

In [ ]:
# Settings Beginning date of the data is 01/07/2020 End date is 29/06/2023
pvd = vtuIO.PVDIO("Results_open_por1.pvd", dim=2)
col_number = len(indices)
start = '7/1/2020 00:00:00'
end = '6/29/2023 00:00:00'

# prepare a data frame
index = pd.date_range(start=start, end=end,  freq="12h")
columns = [f'y{i}' for i in range(col_number)] 
df = pd.DataFrame(index=index, columns=columns)
displacement = pvd.read_time_series("displacement", pts=coords)

# fill in the data
for n, col in enumerate(df.columns):
     df[col] = np.array([displacement[keys[n]][:,0][i] for i in range(len(df))])

# drawing a heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

ax1.plot(df)
#ax1.legend(df.columns)

ax2.imshow(df.T, aspect='auto', cmap='plasma')
#ax2.set_yticks(range(len(df.columns)))
#ax2.set_yticklabels(df.columns)

plt.show()

In [ ]:
# Settings Beginning date of the data is 01/07/2020 End date is 29/06/2023
pvd = vtuIO.PVDIO("Results_open_por1.pvd", dim=2)
col_number = len(indices)
start = '7/1/2020 00:00:00'
end = '6/29/2023 00:00:00'

# prepare a data frame
index = pd.date_range(start=start, end=end,  freq="12h")
columns = [f'y{i}' for i in range(col_number)] 
df = pd.DataFrame(index=index, columns=columns)

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigma_S = pvd.read_time_series("sigma", pts=coords)
sigma_I = pvd.read_time_series("sigma_ice", pts=coords)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=coords)

# fill in the data
for n, col in enumerate(df.columns):
     df[col] = np.array([((sigma_S[keys[n]][:,0] + np.multiply(phi_I[keys[n]],sigma_I[keys[n]][:,0]))*10**(-7))[i] for i in range(len(df))])

# drawing a heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

ax1.plot(df)
#ax1.legend(df.columns)

ax2.imshow(df.T, aspect='auto', cmap='plasma')
#ax2.set_yticks(range(len(df.columns)))
#ax2.set_yticklabels(df.columns)

plt.show()

In [ ]:
# Settings Beginning date of the data is 01/07/2020 End date is 29/06/2023
pvd = vtuIO.PVDIO("Results_open_TEe3.pvd", dim=2)
col_number = len(indices)
start = '7/1/2020 00:00:00'
end = '6/29/2023 00:00:00'

# prepare a data frame
index = pd.date_range(start=start, end=end,  freq="12h")
columns = [f'y{i}' for i in range(col_number)] 
df = pd.DataFrame(index=index, columns=columns)
displacement = pvd.read_time_series("displacement", pts=coords)

# fill in the data
for n, col in enumerate(df.columns):
     df[col] = np.array([displacement[keys[n]][:,0][i] for i in range(len(df))])

# drawing a heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

ax1.plot(df)
#ax1.legend(df.columns)

ax2.imshow(df.T, aspect='auto', cmap='plasma')
#ax2.set_yticks(range(len(df.columns)))
#ax2.set_yticklabels(df.columns)

plt.show()

In [ ]:
# Settings Beginning date of the data is 01/07/2020 End date is 29/06/2023
pvd = vtuIO.PVDIO("Results_open_TEe3.pvd", dim=2)
col_number = len(indices)
start = '7/1/2020 00:00:00'
end = '6/29/2023 00:00:00'

# prepare a data frame
index = pd.date_range(start=start, end=end,  freq="12h")
columns = [f'y{i}' for i in range(col_number)] 
df = pd.DataFrame(index=index, columns=columns)

pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigma_S = pvd.read_time_series("sigma", pts=coords)
sigma_I = pvd.read_time_series("sigma_ice", pts=coords)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=coords)

# fill in the data
for n, col in enumerate(df.columns):
     df[col] = np.array([((sigma_S[keys[n]][:,0] + np.multiply(phi_I[keys[n]],sigma_I[keys[n]][:,0]))*10**(-7))[i] for i in range(len(df))])

# drawing a heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

ax1.plot(df)
#ax1.legend(df.columns)

ax2.imshow(df.T, aspect='auto', cmap='plasma')
#ax2.set_yticks(range(len(df.columns)))
#ax2.set_yticklabels(df.columns)

plt.show()

# Robin B.C.

In the following we study the system with a Robin boundary condition imposed for displacement $u$ on the edge at $r=R$. This would (hopefully) more closely resemble a finite mass of soil which resists the expansion or shrinkage of the plot under study.

The parameter varied here is $\alpha$ which can be considered equivalent to a spring constant. 
$$ \dfrac{\partial u}{\partial r} = \alpha (u_0 - u(r)), \quad \text{for} \ r = R, $$
where for the following we have set $u_0 = 0$.

The tuning of this parameter to represent the real-world scenario under study here is nontrivial, but it serves as a "midpoint" between the more unrealistic cases of the free and fully constrained boundary conditions. However, varying this parameter would also provide us with interesting point of theoretical study into the mechanics of frozen soils and to the case of frost quakes, especially if we link it firmly to lab-testable soil properties such as Young's modulus via Hooke's law.

In [ ]:
#        [(r, z, theta)]                    [0, 0.1, 0.2, ..., 10]
raxis =  [(i,26,0) for i in np.linspace(start=0, stop=2.6, num=100)]
doprint = False

In [ ]:
# Sweeping over different values of alpha ("spring constant") for the Robin b.c.
# Plotting here done for a set time labeled tTime, which is here a constant generated by the Harrison-Stetson method

paraName = "alpha"
paraSatz = np.linspace(0.5e6,50e6,5) #Note: LARGE values of alpha
doprint = True
tTime = 5000

fig, ax = plt.subplots(figsize=(24,12))
fig, ax2 = plt.subplots(figsize=(24,12))

for p in paraSatz:
    if doprint: print("Generating input for " + paraName + "= ", p)
    ofile="../Column_RectMesh_axial_" + paraName + str(int(p)) + ".prj"
    model = ogs.OGS(INPUT_FILE="../Column_RectMesh_axial_Robin.prj", PROJECT_FILE=ofile, ogs_mode="silent")
    model.replace_parameter_value(paraName, p)
    model.write_input()

    if doprint: print("Running simulation")
    model.run_model(path=PATH)

    if doprint: print("Getting results")
    pvdR = vtuIO.PVDIO("Results_Robin.pvd", dim=2)
    rCoord = np.array(raxis).T[0]
    sigma_S = pvdR.read_set_data(tTime, 'sigma', pointsetarray=raxis)
    phi_I = pvdR.read_set_data(tTime, 'ice_volume_fraction', pointsetarray=raxis)
    sigma_I = pvdR.read_set_data(tTime, 'sigma_ice', pointsetarray=raxis)
    sigma_tot_rr = sigma_S[:,0] + np.multiply(phi_I,sigma_I[:,0])
    ax.plot(rCoord, sigma_tot_rr, linewidth=3, label='alpha=$%i $ ' %p)
    
    displacement = pvdR.read_set_data(tTime, 'displacement', pointsetarray=raxis)
    ax2.plot(rCoord, displacement[:,0], linewidth=3, label='alpha=$%i $ ' %p)
    ax2.set_ylabel('$u(r)_{rr}$ /m', fontsize=24)
    
    if doprint: print("Removing created input")
    !rm $ofile

setupPlot(ax)
setupPlot(ax2)

In [ ]:
# Doing the same but gor small values

paraName = "alpha"
paraSatz = np.linspace(0.5e-6,5e1,3) # Note: small values of alpha
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

# Comparing the results to open bcs
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])
time = pvd.timesteps / 3600 # time in h

ax.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='OPEN r=0.01m')
ax.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='OPEN r=R')

for p in paraSatz:
    print("Generating input for " + paraName + "= ", p)
    ofile="../Column_RectMesh_axial_" + paraName + str(int(p)) + ".prj"
    model = ogs.OGS(INPUT_FILE="../Column_RectMesh_axial_Robin.prj", PROJECT_FILE=ofile, ogs_mode="silent")
    model.replace_parameter_value(paraName, p)
    model.write_input()

    print("Running simulation")
    model.run_model(path=PATH)

    pvdR = vtuIO.PVDIO("Results_Robin.pvd", dim=2)
    sigma_S = pvdR.read_time_series("sigma", pts=pts)
    phi_I = pvdR.read_time_series('ice_volume_fraction', pts=pts)
    sigma_I = pvdR.read_time_series('sigma_ice', pts=pts)
    sigma_tot_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
    time = pvdR.timesteps / 3600
    
    ax.plot(time, sigma_tot_rr, linewidth=3, label='alpha=$%i $ ' %p)
    
    #!rm $ofile

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel(r'$\sigma_{rr}$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
dd

In [ ]:
# Different values again
paraName = "alpha"
paraSatz = np.linspace(0.5e-6,5e7,7)
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

# Comparing the results to open bcs
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])
time = pvd.timesteps / 3600 # time in h

ax.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='OPEN r=0.01m')
ax.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='OPEN r=R')

for p in paraSatz:
    print("Generating input for " + paraName + "= ", p)
    ofile="../Column_RectMesh_axial_" + paraName + str(int(p)) + ".prj"
    model = ogs.OGS(INPUT_FILE="../Column_RectMesh_axial_Robin.prj", PROJECT_FILE=ofile, ogs_mode="silent")
    model.replace_parameter_value(paraName, p)
    model.write_input()

    print("Running simulation")
    model.run_model(path=PATH)

    pvdR = vtuIO.PVDIO("Results_Robin.pvd", dim=2)
    sigma_S = pvdR.read_time_series("sigma", pts=pts)
    phi_I = pvdR.read_time_series('ice_volume_fraction', pts=pts)
    sigma_I = pvdR.read_time_series('sigma_ice', pts=pts)
    sigma_tot_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
    time = pvdR.timesteps / 3600
    
    ax.plot(time, sigma_tot_rr, linewidth=3, label='alpha=$%i $ ' %p)
    
    !rm $ofile

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel(r'$\sigma_{rr}$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# And again
paraName = "alpha"
paraSatz = np.linspace(5e7,5e10,3)
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

# Comparing the results to open and closed bcs
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])
time = pvd.timesteps / 3600 # time in h

ax.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='OPEN r=0.01m')
#ax.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='OPEN r=R')

pvd = vtuIO.PVDIO("Results_closed.pvd", dim=2)
sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
sigma_tot1_rr = sigma_S['pt1'][:,0] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,0])
time = pvd.timesteps / 3600 # time in h

ax.plot(time, sigma_tot0_rr, linewidth=3, linestyle='solid', label='CLOSED r=0.01m')
#ax.plot(time, sigma_tot1_rr, linewidth=3, linestyle='solid', label='CLOSED r=R')

for p in paraSatz:
    print("Generating input for " + paraName + "= ", p)
    ofile="../Column_RectMesh_axial_" + paraName + str(int(p)) + ".prj"
    model = ogs.OGS(INPUT_FILE="../Column_RectMesh_axial_Robin.prj", PROJECT_FILE=ofile, ogs_mode="silent")
    model.replace_parameter_value(paraName, p)
    model.write_input()

    print("Running simulation")
    model.run_model(path=PATH)

    pvdR = vtuIO.PVDIO("Results_Robin.pvd", dim=2)
    sigma_S = pvdR.read_time_series("sigma", pts=pts)
    phi_I = pvdR.read_time_series('ice_volume_fraction', pts=pts)
    sigma_I = pvdR.read_time_series('sigma_ice', pts=pts)
    sigma_tot_rr = sigma_S['pt0'][:,0] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,0])
    time = pvdR.timesteps / 3600
    
    ax.plot(time, sigma_tot_rr, linewidth=3, label='alpha=$%i $ ' %p)
    
    !rm $ofile

ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel(r'$\sigma_{rr}$ /Pa', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

pts = {"pt0": (1.25,25.9,0.0), "pt1": (1.25,25.0,0.0)}

results = {}
pvd = vtuIO.PVDIO("Results_Robin.pvd", dim=2)
T_Kelv = pvd.read_time_series("temperature", pts=pts)
time = pvd.timesteps / 3600 # time in h
T_Cels = T_Kelv['pt0'] - 273.15

ax.plot(time, 0*T_Cels, linewidth=2, linestyle='dashed', color='black')
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='lightcoral', label='d=0.1m')

T_Cels = T_Kelv['pt1'] - 273.15
ax.plot(time, T_Cels, linewidth=3, linestyle='solid', color='teal', label='d=1.0m')  

ax.plot(dfA[0], dfA[1], linewidth=1, linestyle='dotted', color='indigo', label='observed 0.1m')
ax.plot(dfATS[0], dfATS[1], linewidth=1.5, linestyle='solid', color='darkslateblue', label='ATS at 1.0m')


ax.set_xlabel('$t$ / h', fontsize=24)
ax.set_ylabel('$T$ /°C', fontsize=24)
ax.grid()
ax.legend(fontsize=24)

In [ ]:
# Looking at the stress (zz) and displacement (zz) 0.1m under the surface close to the origin and at r = R
pts = {"pt0": (0.01,25.9,0.0), "pt1": (2.6,25.9,0.0)}

fig, ax1 = plt.subplots(figsize=(24,12))
fig, ax2 = plt.subplots(figsize=(24,12))

sigma_S = pvd.read_time_series("sigma", pts=pts)
sigma_I = pvd.read_time_series("sigma_ice", pts=pts)
phi_I = pvd.read_time_series("ice_volume_fraction", pts=pts)
sigma_tot0_zz = sigma_S['pt0'][:,1] + np.multiply(phi_I['pt0'],sigma_I['pt0'][:,1])
sigma_tot1_zz = sigma_S['pt1'][:,1] + np.multiply(phi_I['pt1'],sigma_I['pt1'][:,1])

displacement = pvd.read_time_series("displacement", pts=pts)
time = pvd.timesteps / 3600 # time in h

ax1.plot(time, sigma_tot0_zz, linewidth=3, linestyle='solid', label='r=0.01m')
ax1.plot(time, sigma_tot1_zz, linewidth=3, linestyle='solid', label='r=R')

ax1.set_xlabel('$t$ / h', fontsize=24)
ax1.set_ylabel('$\sigma_{zz}$ /Pa', fontsize=24)
ax1.grid()
ax1.legend(fontsize=24)

ax2.plot(time, displacement['pt0'][:,0], linewidth=3, linestyle='solid', label='r=0.01m')
ax2.plot(time, displacement['pt1'][:,0], linewidth=3, linestyle='solid', label='r=R')

ax2.set_xlabel('$t$ / h', fontsize=24)
ax2.set_ylabel('$u_{zz}$ /m', fontsize=24)
ax2.grid()
ax2.legend(fontsize=24)

In [ ]:
events = np.array([20952,21456+1200,22032,22056,22080,22104,22680,22704,22728,23448,23472,23496,23520,23544])*3600
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
zaxis =  [(0.5,i,0) for i in np.linspace(start=0, stop=26, num=100)]
zaxisinv =  [(0.5,26-i,0) for i in np.linspace(start=0, stop=26, num=100)]
temps = {}
for i in range(0,13):
    temps[i] = pvd.read_set_data(events[i], "temperature", pointsetarray=zaxis)-273.15
fig, ax1 = plt.subplots(figsize=(24,12))
for i in range(0,13):
    ax1.plot(zaxisinv,temps[i])
plt.show()

In [ ]:
import numpy as np
events = np.array([20952,21456+1200,22032,22056,22080,22104,22680,22704,22728,23448,23472,23496,23520,23544])*3600
pvd = vtuIO.PVDIO("Results_open.pvd", dim=2)
sigmarr = {}; disps ={}; phiIs={}; 
for i in range(0,13):
    sigma_S = pvd.read_set_data(events[i],"sigma", pointsetarray=zaxis)
    sigma_I = pvd.read_set_data(events[i], "sigma_ice", pointsetarray=zaxis)
    phi_I = pvd.read_set_data(events[i], "ice_volume_fraction", pointsetarray=zaxis)
    sigma_tot0_rr = sigma_S[:,0] + np.multiply(phi_I,sigma_I[:,0])
    sigma_tot0_zz = sigma_S[:,1] + np.multiply(phi_I,sigma_I[:,1])

    displacement = pvd.read_set_data(events[i],"displacement", pointsetarray=zaxis)
    sigmarr[i] = sigma_tot0_rr
    disps[i] = displacement
    phiIs[i] = phi_I
fig, ax1 = plt.subplots(figsize=(24,12))
for i in range(0,13):
    ax1.plot(zaxisinv, sigmarr[i])
plt.show()